In [ ]:
from datetime import date, timedelta, datetime

# Reporting date range start dates
start_dates = [
    date(2025, 1, 1)
]

# Exclusion date ranges - this should be a list of tuples in which each tuple has two
# members, a start date and time and end date and time in that order
exclusions = [
]

# Whether to export the data
export_spreadsheet = True
export_html_dashboard = True

In [ ]:
%run ../api.ipynb
%run ../config.ipynb
%run ../export.ipynb

In [ ]:
DISPLAY_COLS = {
    "sensor_id": "ID",
    "mean": "Mean",
    "cv": "CV",
    "rmssd_norm": "RMSSD",
    "tv_norm": "TV",
    "tir": "TIR",
    "stability_index": "Stab."
}

## CGM Stability Metrics Calculation from Raw Data

In [ ]:
def cgm_stability_metrics(df):
    g = df["level"]
    mean = g.mean()
    return {
        "sensor_id": min(df["date"]).strftime("%y%m%d"),
        "start": min(df["date"]),
        "end": max(df["date"]),
        "mean": mean,
        "cv": g.std() / mean,
        "rmssd_norm": (g.diff()**2).mean()**0.5 / mean,
        "tv_norm": g.diff().abs().sum() / len(g),
        "tir": ((g >= 3.9) & (g <= 10.0)).mean()
    }

## Individual Sensor Data Concatenation

In [ ]:
import pandas as pd

def build_combined_cgm_df(measurements_list, metrics_list):
    frames = []
    for df, meta in zip(measurements_list, metrics_list):
        sensor_id = meta["sensor_id"]

        tmp = df.copy()
        tmp["date"] = pd.to_datetime(tmp["date"])
        tmp["sensor_id"] = sensor_id
        tmp = tmp.sort_values("date")

        # Derived columns
        tmp["time_of_day"] = tmp["date"].dt.hour * 60 + tmp["date"].dt.minute
        tmp["day"] = tmp["date"].dt.date

        frames.append(tmp[["sensor_id", "date", "day", "time_of_day", "level"]])

    combined_df = pd.concat(frames, ignore_index=True)
    return combined_df

## Stability Index and Ranking

In [ ]:

import numpy as np

def add_rankings(metrics_df):
    df = metrics_df.copy()

    # Guardrails
    for col in ["mean", "cv", "rmssd_norm", "tv_norm", "tir"]:
        if col not in df.columns:
            df[col] = np.nan

    # Practical composite (higher = better)
    df["stability_index"] = 1 / (
        df["cv"] +
        df["rmssd_norm"] +
        (df["tv_norm"] / df["mean"]).replace([np.inf, -np.inf], np.nan)
    )

    # Simple ranks (1 = best)
    df["rank_cv"] = df["cv"].rank(method="min")
    df["rank_stability"] = df["stability_index"].rank(ascending=False, method="min")
    df["rank_tir"] = df["tir"].rank(ascending=False, method="min")

    return df

## Per-Sensor AGP Summary

In [ ]:
import pandas as pd

def agp_summary(cgm_long, sensor_id, bin_minutes=15):
    """
    Returns AGP-like summary for one sensor:
    median, p25, p75 by time-of-day bins.
    """
    df = cgm_long[cgm_long["sensor_id"] == sensor_id].copy()
    if df.empty:
        return None

    # bin time-of-day
    df["tod_bin"] = (df["time_of_day"] // bin_minutes) * bin_minutes

    g = df.groupby("tod_bin")["level"]
    out = pd.DataFrame({
        "tod_bin": g.median().index.astype(int),
        "median": g.median().values,
        "p25": g.quantile(0.25).values,
        "p75": g.quantile(0.75).values,
    })

    # Convert bin (minutes) to HH:MM label for hover
    out["hh"] = (out["tod_bin"] // 60).astype(int)
    out["mm"] = (out["tod_bin"] % 60).astype(int)
    out["label"] = out["hh"].astype(str).str.zfill(2) + ":" + out["mm"].astype(str).str.zfill(2)

    return out

## Sensor Comparison Dashboard

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def sensor_comparison_dashboard(metrics_df, cgm_df, top_n=8):
    """
    Creates a multi-panel Plotly dashboard comparing CGM data gathered by multiple sensors
    """
    dfm = metrics_df.reset_index()
    dfm_date_ordered = dfm.sort_values("start")

    # Pick a manageable subset of sensors for some of the comparison plots
    best = dfm.sort_values("stability_index", ascending=False).head(top_n)["sensor_id"].tolist()
    worst = dfm.sort_values("stability_index", ascending=True).head(top_n)["sensor_id"].tolist()
    showcase = list(dict.fromkeys(best + worst))

    fig = make_subplots(
        rows=3, cols=2,
        specs=[
            [{"type": "table"}, {"type": "xy", "secondary_y": True}],
            [{"type": "xy"}, {"type": "xy"}],
            [{"type": "xy"}, {"type": "xy"}],
        ],
        subplot_titles=(
            "Sensor Metrics Table",
            "CV",
            "Mean vs CV (Colour By TIR)",
            "Stability Index",
            "Example Traces (Best vs Worst)",
            "AGP Overlay (Median + IQR over 24h bins)",
        ),
        vertical_spacing=0.12,
        horizontal_spacing=0.08
    )

    # --- Table (top-left) ---
    table_cols = [
        "sensor_id",
        "mean",
        "cv",
        "rmssd_norm",
        "tv_norm",
        "tir",
        "stability_index"
    ]

    present_cols = [c for c in table_cols if c in dfm.columns]

    table_df = dfm[present_cols].copy()

    # rename for display only
    table_df = table_df.rename(columns=DISPLAY_COLS)

    # Formatting
    for c in ["Mean", "CV", "RMSSD", "TV", "TIR", "Stab."]:
        if c in table_df.columns:
            table_df[c] = table_df[c].map(
                lambda x: "" if pd.isna(x) else f"{x:.3f}"
            )
    
    # % formatting for TIR
    table_df["TIR"] = table_df["TIR"].map(
        lambda x: "" if x == "" else f"{float(x) * 100:.1f}%"
    )

    fig.add_trace(
        go.Table(
            header=dict(
                values=list(table_df.columns),
                font=dict(size=11)
            ),
            cells=dict(
                values=[table_df[c].tolist() for c in table_df.columns],
                font=dict(size=10),
                align=["left"] + ["right"] * (len(table_df.columns) - 1)
            )
        ),
        row=1, col=1
    )

    # --- CV bar (top-right) ---
    if "cv" in dfm.columns:
        fig.add_trace(
            go.Bar(x=dfm_date_ordered["sensor_id"], y=dfm_date_ordered["cv"], name="CV"),
            row=1, col=2, secondary_y=False
        )

        # CV = 0.36 threshold line (on the LEFT axis of the CV subplot)
        fig.add_shape(
            type="line",
            x0=0, x1=1,
            xref="x domain",      # span full width of the CV subplot
            y0=0.36, y1=0.36,
            yref="y",             # left y-axis of that subplot
            line=dict(dash="dash", width=1),
        )

        fig.add_annotation(
            x=0, xref="x domain",
            y=0.36, yref="y",
            text="CV = 0.36",
            showarrow=False,
            xanchor="left",
            yanchor="bottom",
        )

        # Mean glucose line (right y-axis)
        fig.add_trace(
            go.Scatter(x=dfm_date_ordered["sensor_id"], y=dfm_date_ordered["mean"], mode="lines+markers", name="Mean"),
            row=1, col=2, secondary_y=True
        )

        fig.update_xaxes(tickangle=45, row=1, col=2)
        fig.update_yaxes(title_text="CV", row=1, col=2, secondary_y=False)
        fig.update_yaxes(title_text="Mean Glucose", row=1, col=2, secondary_y=True)

    # --- Scatter mean vs CV (middle-left) ---
    if all(c in dfm.columns for c in ["mean", "cv", "tir"]):
        scatter = px.scatter(
            dfm,
            x="mean",
            y="cv",
            hover_name="sensor_id",
            hover_data=["tir", "rmssd_norm", "tv_norm", "stability_index"],
            color="tir"
        )

        for tr in scatter.data:
            tr.update(
                marker=dict(
                    size=10,
                    line=dict(width=1, color="black")
                )
            )
            fig.add_trace(tr, row=2, col=1)

    # --- Stability index bar (middle-right) ---
    if "stability_index" in dfm.columns:
        fig.add_trace(
            go.Bar(
                x=dfm_date_ordered["sensor_id"],
                y=dfm_date_ordered["stability_index"],
                name="Stability index"
            ),
            row=2, col=2
        )
        fig.update_xaxes(tickangle=45, row=2, col=2)

    # --- Example traces: 2 best + 2 worst, to maintain clarity ---
    ex_ids = []
    if "stability_index" in dfm.columns:
        ex_ids = (
            dfm.sort_values("stability_index", ascending=False).head(2)["sensor_id"].tolist()
            + dfm.sort_values("stability_index", ascending=True).head(2)["sensor_id"].tolist()
        )
        ex_ids = list(dict.fromkeys(ex_ids))

    for sid in ex_ids:
        d = cgm_df[cgm_df["sensor_id"] == sid].sort_values("date")
        if not d.empty:
            fig.add_trace(
                go.Scatter(x=d["date"], y=d["level"], mode="lines", name=f"{sid} trace"),
                row=3, col=1
            )

    # --- AGP overlay for “showcase” sensors ---
    # Overlaying too many gets too busy; default to best 3 + worst 3
    agp_ids = showcase[:6] if len(showcase) > 6 else showcase
    for sid in agp_ids:
        summ = agp_summary(cgm_df, sid, bin_minutes=15)
        if summ is None:
            continue

        # IQR band
        fig.add_trace(
            go.Scatter(
                x=summ["tod_bin"], y=summ["p75"],
                mode="lines", name=f"{sid} p75",
                showlegend=False, hoverinfo="skip"
            ),
            row=3, col=2
        )

        fig.add_trace(
            go.Scatter(
                x=summ["tod_bin"], y=summ["p25"],
                mode="lines", name=f"{sid} p25",
                fill="tonexty",  # fills down to previous trace
                showlegend=False, hoverinfo="skip"
            ),
            row=3, col=2
        )

        # median
        fig.add_trace(
            go.Scatter(
                x=summ["tod_bin"], y=summ["median"],
                mode="lines", name=f"{sid} median",
                hovertemplate="Time %{x} min<br>Median %{y:.2f}<extra>" + sid + "</extra>"
            ),
            row=3, col=2
        )

    fig.update_xaxes(title_text="Time-of-day (minutes from midnight)", row=3, col=2)
    fig.update_yaxes(title_text="Glucose", row=3, col=2)

    fig.update_layout(
        height=1200,
        title=dict(
            text="CGM Comparison Dashboard",
            x=0.5,
            xanchor="center"
        ),
        legend=dict(
            orientation="h",
            yanchor="top",
            y=-0.07,
            xanchor="center",
            x=0.5
        ),
        margin=dict(t=90, b=120)
    )

    return fig

## Load Per-Sensor Data and Calculate Stability Metrics

In [ ]:
import pandas as pd

# Log in to the service, get the person ID and retrieve the data
token = authenticate(url, username, password)
person_id = get_person_id(url, token, firstnames, surname)

# Iterate over each date
measurements_list = []
metrics_list = []
for start_date in start_dates:
    # Calculate the end date based on sensor lifespan and load the data
    end_date = start_date + timedelta(days=11)
    df = get_blood_glucose_measurements(url, token, person_id, start_date, end_date)

    # Remove any excluded ranges
    df = remove_records_for_date_ranges(df, exclusions)

    # Add this data set to the list
    measurements_list.append(df)

    # Calculate the metrics and append them to the metrics list
    stability_metrics = cgm_stability_metrics(df)
    metrics_list.append(stability_metrics)

# Conver the stability metrics list to a data frame and preview the data
metrics_df = pd.DataFrame(metrics_list)
metrics_df = metrics_df.set_index("sensor_id")
metrics_df = add_rankings(metrics_df)
display(metrics_df)


In [ ]:
# Export the data to a spreadsheet
if export_spreadsheet:
    export_to_spreadsheet("glucose", "glucose_cgm_dashboard", { "Sensor Metrics": metrics_df })

# Combine the individual CGM datasets and plot the dashboard
combined_df = build_combined_cgm_df(measurements_list, metrics_list)
fig = sensor_comparison_dashboard(metrics_df, combined_df, top_n=8)

# Export to HTML
if export_html_dashboard:
    export_dashboard(fig, "glucose", "glucose_cgm_dashboard", "")

fig.show()